# Spoofed Speech Detection via Maximum Likelihood Estimation of Gaussian Mixture Models
The goal of synthetic speech detection is to determine whether a speech segment $S$ is natural or synthetic/converted speeach.

This notebook implements a Gaussian mixture model maximum likelihood (GMM-ML) classifier for synthetic (spoofed) speech detection. This approach uses regular mel frequency cepstral coefficients (MFCC) features and gives the best performance on the [ASVspoof 2015 dataset](https://www.idiap.ch/dataset/avspoof) among the standard classifiers (GMM-SV, GMM-UBM, ...). For more background information see: *Hanilçi, Cemal, Tomi Kinnunen, Md Sahidullah, and Aleksandr Sizov. "Classifiers for synthetic speech detection: a comparison." In INTERSPEECH 2015*. The scripts use the Python package [Bob.Bio.SPEAR 2.04](https://pypi.python.org/pypi/bob.bio.spear/2.0.4) for speaker recogntion.

This work is part of the ["DDoS Resilient Emergency Dispatch Center"](https://www.dhs.gov/science-and-technology/news/2015/09/04/dhs-st-awards-university-houston-26m-cyber-security-research) project at the University of Houston, funded by the Department of Homeland Security (DHS).


April 19, 2015

Lorenzo Rossi

(lorenzo **[dot]** rossi **[at]** gmail **[dot]** com)

In [1]:
import os
import time
import numpy as np
import pandas as pd
import scipy
import cqcc_ext
import cfcc_ext
import librosa
import random
from cqcc import cqcc as CQCC
from sklearn.metrics import classification_report, roc_curve, roc_auc_score

pd.set_option('display.max_rows', 300)

WAV_FOLDER = '/home/wujunting/data/ASVspoof2019/LA/' #'ASV2019dataset/wav/' # Path to folder containing speakers .wav subfolders
LABEL_FOLDER = '/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_cm_protocols/' #'ASV2019dataset/CM_protocol/' # Path to ground truth csv files

evalu_before = 'ASVspoof2019_LA_eval'
train_before = 'ASVspoof2019_LA_train'

EXT = '.wav'
%matplotlib inline

## Loading the Ground Truth
Load the dataframes (tables) with the labels for the training, development and evaluation (hold out) sets. Each subfolder corresponds to a different speaker. For example, T1 and D4 indicate the subfolders associated to the utterances and spoofed segments of speakers T1 and D4, respectively in training and development sets. Note that number of evaluation samples >> number of development samples >> testing samples.

You can either select the speakers in each set one by one, *e.g.*:
```
train_subfls = ['T1', 'T2']
``` 
will only load segments from speakers T1 and T2 for training,

or use all the available speakers in a certain subset by leaving the list empty, *e.g.*:
```
devel_subfls = [] 
```
will load all the available Dx speaker segments for the development stage. If you are running this notebook for the first time, you may want to start only with 2 or so speakers per set for sake of quick testing. All the scripts may take several hours to run on the full size datsets.

In [2]:
train_subfls = ['wav']#, 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T13']  #T13 used instead of T10 for gender balance
devel_subfls = []#, 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10']
evalu_subfls = ['wav']#, 'E2', 'E3', 'E4', 'E5', 'E6','E7',  'E8', 'E9', 'E10']
train = pd.read_csv(LABEL_FOLDER + 'ASVspoof2019.LA.cm.train.trn.txt', sep=' ', header=None, names=['folder','file','default','method','source'])
train.sort_values(['folder', 'file'], inplace=True)
print(train)
devel = pd.read_csv(LABEL_FOLDER + 'ASVspoof2019.LA.cm.dev.trl.txt', sep=' ', header=None, names=['folder','file','default','method','source'])
devel.sort_values(['folder', 'file'], inplace=True)
#print(devel)
evalu = pd.read_csv(LABEL_FOLDER +'ASVspoof2019.LA.cm.30000_ubm.txt', sep=' ', header=None, names=['folder','file','default','method','source'])
print(evalu)
#evalu.sort_values(['folder', 'file'], inplace=True)

label_2_class = {'bonafide':1, 'spoof':0}

print('training samples:',len(train))
print('development samples:',len(devel))
print('evaluation samples:',len(evalu))

        folder          file default method    source
2580   LA_0079  LA_T_1004644       -    A01     spoof
13980  LA_0079  LA_T_1006715       -    A04     spoof
10180  LA_0079  LA_T_1007408       -    A03     spoof
10181  LA_0079  LA_T_1017386       -    A03     spoof
6380   LA_0079  LA_T_1025525       -    A02     spoof
...        ...           ...     ...    ...       ...
13979  LA_0098  LA_T_9962675       -    A03     spoof
25379  LA_0098  LA_T_9982036       -    A06     spoof
21579  LA_0098  LA_T_9983869       -    A05     spoof
17779  LA_0098  LA_T_9984124       -    A04     spoof
2579   LA_0098  LA_T_9987202       -      -  bonafide

[25380 rows x 5 columns]
        folder          file default method source
0      LA_0039  LA_E_2834763       -    A11  spoof
1      LA_0014  LA_E_8877452       -    A14  spoof
2      LA_0040  LA_E_6828287       -    A16  spoof
3      LA_0022  LA_E_6977360       -    A09  spoof
4      LA_0031  LA_E_5932896       -    A13  spoof
...        ...      

## Speech Preprocessing and MFCC Extraction
Silence removal and MFCC feature extraction for training segments. More details about the bob.bio.spear involved libraries at:
https://www.idiap.ch/software/bob/docs/latest/bioidiap/bob.bio.spear/master/implemented.html

You can also skip this stage and load a set of feaures (see **Loading features** cell).

In [3]:
# Parameters
 # number of ceptral coefficients (implicit in extractor)
silence_removal_ratio = .1

In [4]:
subfolders = evalu_subfls
ground_truth = train
des = []
des1 = []

# initialize feature matrix
cqt = []
logcqt = []


y_list = []
#y = np.zeros((len(ground_truth),))
print("Extracting features for training stage.")

#vad = preprocessor.Energy_Thr(ratio_threshold=silence_removal_ratio)
#cepstrum = extractor.Cepstral()

k = 0
start_time = time.clock()
length = 0

n_subfls = len(subfolders)
for folder in subfolders[0:n_subfls]:
    #print(folder, end=", ")
    train_folder = "".join([WAV_FOLDER,train_before,'/'])
    folder = "".join((train_folder,folder,'/'))
    #print(folder)
    f_list = os.listdir(folder)
    # print(f_list)
    for f_name in f_list:
        #print(f_name)
        if f_name.endswith(EXT):
        # ground truth
            try:
                # 根据train中file(路径)字段，提取相应.wav文件的source(spoof or human)字段
                label = ground_truth[ground_truth.file==f_name[0:12]].source.values[0]
                #des1.append(ground_truth[ground_truth.file==f_name[0:12]].method.values[0])
                #print(len(des1))
                #print(ground_truth[ground_truth.file==f_name[0:12]].method.values[0])
                
            except IndexError:
                continue
            #print(label)
            y_list.append(label_2_class[label])
            # silence removal
            # print(vad)
            #rate,x = cqcc_ext.read(folder+f_name)
            #vad_data = cfcc_ext.vad(x)
            # CFCC extraction 
            #cqcc_feature = cqcc_ext.cqcc(vad_data,rate)
            #features.append(cqcc_feature)
            #k += 1
            
            
            
            #y_list.append(label_2_class[label])
        # silence removal
            #print(f_name)
        # print(vad)
            x,fs = librosa.load(folder+f_name)
            # xf = x.reshape(x.shape[0], 1)
            #print(x)
            B = 96
            #fmax = fs/2
            #fmin = fmax/2**9
            d = 16
            cf = 19
            ZsdD = 'ZsdD'
            k+=1
            print(k)
            #C = np.abs(librosa.cqt(x, sr=16000, fmin=librosa.note_to_hz('C2'),n_bins=60))
            
            
            CQcc = np.transpose(np.abs(librosa.stft(x)))
            CQcc_orig  = CQcc
            if CQcc.shape[0] > 130:
                random_po = random.randrange(0, CQcc.shape[0]-130)
                CQcc = CQcc[:130, :]
            elif CQcc.shape[0] < 130:
                CQcc_orig = CQcc
                make_up = int(130/CQcc.shape[0])
                for i in range(make_up-1):
                       #print(i)
                    CQcc = np.vstack((CQcc,CQcc_orig))
                CQcc = np.vstack((CQcc, CQcc_orig[:(130-(CQcc_orig.shape[0]*make_up)), :]))
            print(CQcc.shape)
            cqt.append(CQcc)
            #logcqt.append(np.transpose(S))

            # MFCC extraction 
            #CQcc, LogP_absCQT, TimeVec, FreqVec, Ures_LogP_absCQT, Ures_FreqVec, absCQT = CQCC(xf, fs, B, fmax, fmin, d, cf, ZsdD)
            #print(CQcc.shape)
            #print("????????????")
            #cqt.append(absCQT)
            #logcqt.append(LogP_absCQT)
            #cqcc_orig.append(CQcc[:, :20])
            #cqcc_one.append(CQcc[:, 20:40])
            #cqcc_two.append(CQcc[:, 40:60])
            #cqcc_orig_one.append(CQcc[:, :40])
            #print(features[:, :40].shape)
            #print("done")

# 这里是将人声与合成声进行了区分的，根据就是y
#y = np.array(y_list)
#Xf_cqcc = np.array(cqcc)
#Xf_cqcc_orig = np.array(cqcc_orig)
#Xf_cqcc_one = np.array(cqcc_one)
#Xf_cqcc_two = np.array(cqcc_two)
#Xf_cqcc_orig_one = np.array(cqcc_orig_one)
#print(k,"files processed in",(time.clock()-start_time)/60,"minutes.")
#np.save('wavenet_Xf_from_trainset_500.npy',Xf_cqcc)
#np.save('Xf_cqcc_orig.npy',Xf_cqcc_orig)
#np.save('Xf_cqcc_one.npy',Xf_cqcc_one)
#np.save('Xf_cqcc_two.npy',Xf_cqcc_two)
#np.save('Xf_cqcc_orig_one.npy',Xf_cqcc_orig_one)
#np.save('y_cqcc.npy',y_list)
#print('Feature and label matrices saved to disk')

Extracting features for training stage.


/home/wujunting/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


1
(130, 1025)
2
(130, 1025)
3
(130, 1025)
4
(130, 1025)
5
(130, 1025)
6
(130, 1025)
7
(130, 1025)
8
(130, 1025)
9
(130, 1025)
10
(130, 1025)
11
(130, 1025)
12
(130, 1025)
13
(130, 1025)
14
(130, 1025)
15
(130, 1025)
16
(130, 1025)
17
(130, 1025)
18
(130, 1025)
19
(130, 1025)
20
(130, 1025)
21
(130, 1025)
22
(130, 1025)
23
(130, 1025)
24
(130, 1025)
25
(130, 1025)
26
(130, 1025)
27
(130, 1025)
28
(130, 1025)
29
(130, 1025)
30
(130, 1025)
31
(130, 1025)
32
(130, 1025)
33
(130, 1025)
34
(130, 1025)
35
(130, 1025)
36
(130, 1025)
37
(130, 1025)
38
(130, 1025)
39
(130, 1025)
40
(130, 1025)
41
(130, 1025)
42
(130, 1025)
43
(130, 1025)
44
(130, 1025)
45
(130, 1025)
46
(130, 1025)
47
(130, 1025)
48
(130, 1025)
49
(130, 1025)
50
(130, 1025)
51
(130, 1025)
52
(130, 1025)
53
(130, 1025)
54
(130, 1025)
55
(130, 1025)
56
(130, 1025)
57
(130, 1025)
58
(130, 1025)
59
(130, 1025)
60
(130, 1025)
61
(130, 1025)
62
(130, 1025)
63
(130, 1025)
64
(130, 1025)
65
(130, 1025)
66
(130, 1025)
67
(130, 1025)
68
(

522
(130, 1025)
523
(130, 1025)
524
(130, 1025)
525
(130, 1025)
526
(130, 1025)
527
(130, 1025)
528
(130, 1025)
529
(130, 1025)
530
(130, 1025)
531
(130, 1025)
532
(130, 1025)
533
(130, 1025)
534
(130, 1025)
535
(130, 1025)
536
(130, 1025)
537
(130, 1025)
538
(130, 1025)
539
(130, 1025)
540
(130, 1025)
541
(130, 1025)
542
(130, 1025)
543
(130, 1025)
544
(130, 1025)
545
(130, 1025)
546
(130, 1025)
547
(130, 1025)
548
(130, 1025)
549
(130, 1025)
550
(130, 1025)
551
(130, 1025)
552
(130, 1025)
553
(130, 1025)
554
(130, 1025)
555
(130, 1025)
556
(130, 1025)
557
(130, 1025)
558
(130, 1025)
559
(130, 1025)
560
(130, 1025)
561
(130, 1025)
562
(130, 1025)
563
(130, 1025)
564
(130, 1025)
565
(130, 1025)
566
(130, 1025)
567
(130, 1025)
568
(130, 1025)
569
(130, 1025)
570
(130, 1025)
571
(130, 1025)
572
(130, 1025)
573
(130, 1025)
574
(130, 1025)
575
(130, 1025)
576
(130, 1025)
577
(130, 1025)
578
(130, 1025)
579
(130, 1025)
580
(130, 1025)
581
(130, 1025)
582
(130, 1025)
583
(130, 1025)
584
(130

1034
(130, 1025)
1035
(130, 1025)
1036
(130, 1025)
1037
(130, 1025)
1038
(130, 1025)
1039
(130, 1025)
1040
(130, 1025)
1041
(130, 1025)
1042
(130, 1025)
1043
(130, 1025)
1044
(130, 1025)
1045
(130, 1025)
1046
(130, 1025)
1047
(130, 1025)
1048
(130, 1025)
1049
(130, 1025)
1050
(130, 1025)
1051
(130, 1025)
1052
(130, 1025)
1053
(130, 1025)
1054
(130, 1025)
1055
(130, 1025)
1056
(130, 1025)
1057
(130, 1025)
1058
(130, 1025)
1059
(130, 1025)
1060
(130, 1025)
1061
(130, 1025)
1062
(130, 1025)
1063
(130, 1025)
1064
(130, 1025)
1065
(130, 1025)
1066
(130, 1025)
1067
(130, 1025)
1068
(130, 1025)
1069
(130, 1025)
1070
(130, 1025)
1071
(130, 1025)
1072
(130, 1025)
1073
(130, 1025)
1074
(130, 1025)
1075
(130, 1025)
1076
(130, 1025)
1077
(130, 1025)
1078
(130, 1025)
1079
(130, 1025)
1080
(130, 1025)
1081
(130, 1025)
1082
(130, 1025)
1083
(130, 1025)
1084
(130, 1025)
1085
(130, 1025)
1086
(130, 1025)
1087
(130, 1025)
1088
(130, 1025)
1089
(130, 1025)
1090
(130, 1025)
1091
(130, 1025)
1092
(130, 102

...

23460
(130, 1025)
23461
(130, 1025)
23462
(130, 1025)
23463
(130, 1025)
23464
(130, 1025)
23465
(130, 1025)
23466
(130, 1025)
23467
(130, 1025)
23468
(130, 1025)
23469
(130, 1025)
23470
(130, 1025)
23471
(130, 1025)
23472
(130, 1025)
23473
(130, 1025)
23474
(130, 1025)
23475
(130, 1025)
23476
(130, 1025)
23477
(130, 1025)
23478
(130, 1025)
23479
(130, 1025)
23480
(130, 1025)
23481
(130, 1025)
23482
(130, 1025)
23483
(130, 1025)
23484
(130, 1025)
23485
(130, 1025)
23486
(130, 1025)
23487
(130, 1025)
23488
(130, 1025)
23489
(130, 1025)
23490
(130, 1025)
23491
(130, 1025)
23492
(130, 1025)
23493
(130, 1025)
23494
(130, 1025)
23495
(130, 1025)
23496
(130, 1025)
23497
(130, 1025)
23498
(130, 1025)
23499
(130, 1025)
23500
(130, 1025)
23501
(130, 1025)
23502
(130, 1025)
23503
(130, 1025)
23504
(130, 1025)
23505
(130, 1025)
23506
(130, 1025)
23507
(130, 1025)
23508
(130, 1025)
23509
(130, 1025)
23510
(130, 1025)
23511
(130, 1025)
23512
(130, 1025)
23513
(130, 1025)
23514
(130, 1025)
23515
(130

23917
(130, 1025)
23918
(130, 1025)
23919
(130, 1025)
23920
(130, 1025)
23921
(130, 1025)
23922
(130, 1025)
23923
(130, 1025)
23924
(130, 1025)
23925
(130, 1025)
23926
(130, 1025)
23927
(130, 1025)
23928
(130, 1025)
23929
(130, 1025)
23930
(130, 1025)
23931
(130, 1025)
23932
(130, 1025)
23933
(130, 1025)
23934
(130, 1025)
23935
(130, 1025)
23936
(130, 1025)
23937
(130, 1025)
23938
(130, 1025)
23939
(130, 1025)
23940
(130, 1025)
23941
(130, 1025)
23942
(130, 1025)
23943
(130, 1025)
23944
(130, 1025)
23945
(130, 1025)
23946
(130, 1025)
23947
(130, 1025)
23948
(130, 1025)
23949
(130, 1025)
23950
(130, 1025)
23951
(130, 1025)
23952
(130, 1025)
23953
(130, 1025)
23954
(130, 1025)
23955
(130, 1025)
23956
(130, 1025)
23957
(130, 1025)
23958
(130, 1025)
23959
(130, 1025)
23960
(130, 1025)
23961
(130, 1025)
23962
(130, 1025)
23963
(130, 1025)
23964
(130, 1025)
23965
(130, 1025)
23966
(130, 1025)
23967
(130, 1025)
23968
(130, 1025)
23969
(130, 1025)
23970
(130, 1025)
23971
(130, 1025)
23972
(130

24375
(130, 1025)
24376
(130, 1025)
24377
(130, 1025)
24378
(130, 1025)
24379
(130, 1025)
24380
(130, 1025)
24381
(130, 1025)
24382
(130, 1025)
24383
(130, 1025)
24384
(130, 1025)
24385
(130, 1025)
24386
(130, 1025)
24387
(130, 1025)
24388
(130, 1025)
24389
(130, 1025)
24390
(130, 1025)
24391
(130, 1025)
24392
(130, 1025)
24393
(130, 1025)
24394
(130, 1025)
24395
(130, 1025)
24396
(130, 1025)
24397
(130, 1025)
24398
(130, 1025)
24399
(130, 1025)
24400
(130, 1025)
24401
(130, 1025)
24402
(130, 1025)
24403
(130, 1025)
24404
(130, 1025)
24405
(130, 1025)
24406
(130, 1025)
24407
(130, 1025)
24408
(130, 1025)
24409
(130, 1025)
24410
(130, 1025)
24411
(130, 1025)
24412
(130, 1025)
24413
(130, 1025)
24414
(130, 1025)
24415
(130, 1025)
24416
(130, 1025)
24417
(130, 1025)
24418
(130, 1025)
24419
(130, 1025)
24420
(130, 1025)
24421
(130, 1025)
24422
(130, 1025)
24423
(130, 1025)
24424
(130, 1025)
24425
(130, 1025)
24426
(130, 1025)
24427
(130, 1025)
24428
(130, 1025)
24429
(130, 1025)
24430
(130

24832
(130, 1025)
24833
(130, 1025)
24834
(130, 1025)
24835
(130, 1025)
24836
(130, 1025)
24837
(130, 1025)
24838
(130, 1025)
24839
(130, 1025)
24840
(130, 1025)
24841
(130, 1025)
24842
(130, 1025)
24843
(130, 1025)
24844
(130, 1025)
24845
(130, 1025)
24846
(130, 1025)
24847
(130, 1025)
24848
(130, 1025)
24849
(130, 1025)
24850
(130, 1025)
24851
(130, 1025)
24852
(130, 1025)
24853
(130, 1025)
24854
(130, 1025)
24855
(130, 1025)
24856
(130, 1025)
24857
(130, 1025)
24858
(130, 1025)
24859
(130, 1025)
24860
(130, 1025)
24861
(130, 1025)
24862
(130, 1025)
24863
(130, 1025)
24864
(130, 1025)
24865
(130, 1025)
24866
(130, 1025)
24867
(130, 1025)
24868
(130, 1025)
24869
(130, 1025)
24870
(130, 1025)
24871
(130, 1025)
24872
(130, 1025)
24873
(130, 1025)
24874
(130, 1025)
24875
(130, 1025)
24876
(130, 1025)
24877
(130, 1025)
24878
(130, 1025)
24879
(130, 1025)
24880
(130, 1025)
24881
(130, 1025)
24882
(130, 1025)
24883
(130, 1025)
24884
(130, 1025)
24885
(130, 1025)
24886
(130, 1025)
24887
(130

25288
(130, 1025)
25289
(130, 1025)
25290
(130, 1025)
25291
(130, 1025)
25292
(130, 1025)
25293
(130, 1025)
25294
(130, 1025)
25295
(130, 1025)
25296
(130, 1025)
25297
(130, 1025)
25298
(130, 1025)
25299
(130, 1025)
25300
(130, 1025)
25301
(130, 1025)
25302
(130, 1025)
25303
(130, 1025)
25304
(130, 1025)
25305
(130, 1025)
25306
(130, 1025)
25307
(130, 1025)
25308
(130, 1025)
25309
(130, 1025)
25310
(130, 1025)
25311
(130, 1025)
25312
(130, 1025)
25313
(130, 1025)
25314
(130, 1025)
25315
(130, 1025)
25316
(130, 1025)
25317
(130, 1025)
25318
(130, 1025)
25319
(130, 1025)
25320
(130, 1025)
25321
(130, 1025)
25322
(130, 1025)
25323
(130, 1025)
25324
(130, 1025)
25325
(130, 1025)
25326
(130, 1025)
25327
(130, 1025)
25328
(130, 1025)
25329
(130, 1025)
25330
(130, 1025)
25331
(130, 1025)
25332
(130, 1025)
25333
(130, 1025)
25334
(130, 1025)
25335
(130, 1025)
25336
(130, 1025)
25337
(130, 1025)
25338
(130, 1025)
25339
(130, 1025)
25340
(130, 1025)
25341
(130, 1025)
25342
(130, 1025)
25343
(130

In [5]:
y = np.array(y_list)
cqt = np.array(cqt)
#cqt = np.array(cqt)
#print(Xf_cqcc.shape)
#Xf_cqcc_orig = np.array(cqcc_orig)
#print(Xf_cqcc.shape)
#Xf_cqcc_one = np.array(cqcc_one)
#print(Xf_cqcc.shape)
#Xf_cqcc_two = np.array(cqcc_two)
#print(Xf_cqcc.shape)
#Xf_cqcc_orig_one = np.array(cqcc_orig_one)
#print(Xf_cqcc.shape)
np.save('wavenet_y_from_trainset.npy',y)
np.save('wavenet_stft_from_trainset_130.npy',cqt)
#np.save('wavenet_y_from_trainset.npy',y)
#print("done1")
#np.save('Xf_cqcc_orig.npy',Xf_cqcc_orig)
#print("done2")
#np.save('Xf_cqcc_one.npy',Xf_cqcc_one)
#print("done3")
#np.save('Xf_cqcc_two.npy',Xf_cqcc_two)
#print("done4")
#np.save('Xf_cqcc_orig_one.npy',Xf_cqcc_orig_one)
#print("done5")
#np.save('wavenet_y_from_evalset_1000.npy',y_list)
print('Feature and label matrices saved to disk')

Feature and label matrices saved to disk


In [ ]:
import numpy as np
cqcc = np.load('Xf_ubm_cqcc.npy')
from sklearn.mixture import GaussianMixture
print(cqcc.shape)
gmm=GaussianMixture(n_components=512,covariance_type='diag',max_iter=200,n_init=1)
print("done")

In [ ]:
gmm.fit(cqcc)
print("done")

In [34]:
np.vstack(Xf[y==1]).shape

(862811, 60)

In [80]:
ground_truth

,folder,file,default,method,source,z,z_gmm,score_gmm
107206,E2,E200001s,-,spoof,spoof,0,1.0,7.622108
98412,E2,E200002s,-,spoof,spoof,0,1.0,10.544859
102443,E2,E200003h,-,bonafide,bonafide,1,0.0,-2.888099
103712,E2,E200004s,-,spoof,spoof,0,0.0,-2.797664
98984,E2,E200005h,-,bonafide,bonafide,1,1.0,17.829845
...,...,...,...,...,...,...,...,...
105581,E2,E208996h,-,bonafide,bonafide,1,0.0,-3.521843
100661,E2,E208997h,-,bonafide,bonafide,1,0.0,-3.759985
106825,E2,E208998s,-,spoof,spoof,0,1.0,6.707735
102766,E2,E208999h,-,bonafide,bonafide,1,1.0,7.150391


### Saving features

In [18]:
np.save('Xf_cqcc.npy',Xf_mfcc)
np.save('Xf_cqcc_orig.npy',Xf_mfcc_orig)
np.save('Xf_cqcc_one.npy',Xf_mfcc_one)
np.save('Xf_cqcc_two.npy',Xf_mfcc_two)
np.save('Xf_cqcc_orig_one.npy',Xf_mfcc_orig_one)
np.save('y_cqcc.npy',y)
print('Feature and label matrices saved to disk')

Feature and label matrices saved to disk


### Loading features

In [20]:
# Load already extracter features to skip the preprocessing-extraction stage
# Xf = np.load('train_features_10.npy')
# y = np.load('y_10.npy')
Xf = np.load('Xf_cqcc.npy',allow_pickle=True)
y = np.load('y_cqcc.npy',allow_pickle=True)

# GMM - ML Classification

In [22]:
print(Xf.shape)

(25380,)


## GMM Training
Train the GMMs for natural and synthetic speach. For documentation on bob.bio k-means and GMM machines see:
https://pythonhosted.org/bob.learn.em/guide.html

You can also skip the training stage and load an already trained GMM model (see cell **Loading GMM Model**).

In [15]:
# Parameters of the GMM machines
n_gaussians = 512 # number of Gaussians
max_iterats = 25 # maximum number of iterations
n_ceps = 60

### GMM for natural speech 

In [16]:
# Initialize and train k-means machine: the means will initialize EM algorithm for GMM machine
start_time = time.clock()
kmeans_nat = em.KMeansMachine(n_gaussians,n_ceps)
kmeansTrainer = em.KMeansTrainer()
print(kmeans_nat.shape,Xf[y==1].shape)
em.train(kmeansTrainer, kmeans_nat, np.vstack(Xf[y==1]), max_iterations = max_iterats, convergence_threshold = 1e-5)
#kmeans_nat.means

# initialize and train GMM machine
gmm_nat = em.GMMMachine(n_gaussians,n_ceps)
trainer = em.ML_GMMTrainer(True, True, True)
gmm_nat.means = kmeans_nat.means
em.train(trainer, gmm_nat, np.vstack(Xf[y==1]), max_iterations = max_iterats, convergence_threshold = 1e-5)
print("Done in:", (time.clock() - start_time)/60, "minutes")
print(gmm_nat)
gmm_nat.save(HDF5File('pzb_cqcc_nat.hdf5', 'w'))

/home/wujunting/anaconda3/envs/bob_py3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


(512, 60) (2580,)


/home/wujunting/anaconda3/envs/bob_py3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


Done in: 100.58375983333332 minutes


### GMM for synthetic speech

In [5]:
# initialize and train k-means machine: the means will initialize EM algorithm for GMM machine
start_time = time.clock()
kmeans_synt = em.KMeansMachine(n_gaussians,n_ceps)
kmeansTrainer = em.KMeansTrainer()
print(kmeans_synt.shape,Xf[y==0].shape)
em.train(kmeansTrainer, kmeans_synt, np.vstack(Xf[y==0]), max_iterations = max_iterats, convergence_threshold = 1e-5)

# initialize and train GMM machine
gmm_synt = em.GMMMachine(n_gaussians,n_ceps)
trainer = em.ML_GMMTrainer(True, True, True)
gmm_synt.means = kmeans_synt.means
em.train(trainer, gmm_synt, np.vstack(Xf[y==0]), max_iterations = max_iterats, convergence_threshold = 1e-5)
print("Done in:", (time.clock() - start_time)/60, "minutes")
gmm_synt.save(HDF5File('pzb_cqcc_synt.hdf5', 'w'))
print(gmm_synt)

/home/wujunting/anaconda3/envs/bob_py3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


NameError: name 'n_gaussians' is not defined

### Saving GMM model

In [52]:
gmm_nat.save(HDF5File('gmm_nat_500seg.hdf5', 'w'))
gmm_synt.save(HDF5File('gmm_synt_500seg.hdf5', 'w'))

### Loading GMM model

In [3]:
gmm_nat = em.GMMMachine()
gmm_nat.load(HDF5File('pzb_cqcc_nat.hdf5', 'r'))
gmm_synt = em.GMMMachine()
gmm_synt.load(HDF5File('pzb_cqcc_synt.hdf5','r'))
gmm_orig_nat = em.GMMMachine()
gmm_orig_nat.load(HDF5File('pzb_cqcc_orig_nat.hdf5', 'r'))
gmm_orig_synt = em.GMMMachine()
gmm_orig_synt.load(HDF5File('pzb_cqcc_orig_synt.hdf5','r'))
gmm_one_nat = em.GMMMachine()
gmm_one_nat.load(HDF5File('pzb_cqcc_one_nat.hdf5', 'r'))
gmm_one_synt = em.GMMMachine()
gmm_one_synt.load(HDF5File('pzb_cqcc_one_synt.hdf5','r'))
gmm_two_nat = em.GMMMachine()
gmm_two_nat.load(HDF5File('pzb_cqcc_two_nat.hdf5', 'r'))
gmm_two_synt = em.GMMMachine()
gmm_two_synt.load(HDF5File('pzb_cqcc_two_synt.hdf5','r'))
gmm_orig_one_nat = em.GMMMachine()
gmm_orig_one_nat.load(HDF5File('pzb_cqcc_orig_one_nat.hdf5', 'r'))
gmm_orig_one_synt = em.GMMMachine()
gmm_orig_one_synt.load(HDF5File('pzb_cqcc_orig_one_synt.hdf5','r'))

## GMM-ML Scoring
Extract the features for the testing data, compute the likelihood ratio test and  compute ROC AUC and estimated EER scores.

In [6]:
subfolders = train_subfls
ground_truth = train

print(len(ground_truth))
# initialize score and class arrays
gmm_nat_score = np.zeros(len(ground_truth),)
gmm_synt_score = np.zeros(len(ground_truth),)
llr_gmm_score = np.zeros(len(ground_truth),)
z_gmm = np.zeros(len(ground_truth),)

gmm_nat_orig_score = np.zeros(len(ground_truth),)
gmm_synt_orig_score = np.zeros(len(ground_truth),)
llr_gmm_orig_score = np.zeros(len(ground_truth),)
z_orig_gmm = np.zeros(len(ground_truth),)

gmm_nat_one_score = np.zeros(len(ground_truth),)
gmm_synt_one_score = np.zeros(len(ground_truth),)
llr_gmm_one_score = np.zeros(len(ground_truth),)
z_one_gmm = np.zeros(len(ground_truth),)

gmm_nat_two_score = np.zeros(len(ground_truth),)
gmm_synt_two_score = np.zeros(len(ground_truth),)
llr_gmm_two_score = np.zeros(len(ground_truth),)
z_two_gmm = np.zeros(len(ground_truth),)

gmm_nat_orig_one_score = np.zeros(len(ground_truth),)
gmm_synt_orig_one_score = np.zeros(len(ground_truth),)
llr_gmm_orig_one_score = np.zeros(len(ground_truth),)
z_orig_one_gmm = np.zeros(len(ground_truth),)
print(status)

vad = preprocessor.Energy_Thr(ratio_threshold=.1)
cepstrum = extractor.Cepstral()

k = 0
thr = .5

#speaker = ground_truth[ground_truth.folder==speaker_id]
f_list = list(ground_truth.file)
evalu_folder = "".join([WAV_FOLDER,train_before,'/'])
folder = "".join([evalu_folder,evalu_subfls[0],'/'])
#print(f_list)
print(folder)
for f in f_list:
    if isinstance(f,str):
        f_name = "".join([folder,f,EXT])
    if os.path.exists(f_name):
        # silence removal
        # print(f_name)
        # print(vad)
        x,fs = librosa.load(f_name, sr = 16000)
        x = x.reshape(x.shape[0], 1)
        print(f_name)
        B = 96
        fmax = fs/2
        fmin = fmax/2**9
        d = 16
        cf = 19
        ZsdD = 'ZsdD'
        CQcc, LogP_absCQT, TimeVec, FreqVec, Ures_LogP_absCQT, Ures_FreqVec, absCQT = CQCC(x, fs, B, fmax, fmin, d, cf, ZsdD)
        print("orig_shape:", CQcc.shape)
        if CQcc.shape[0] >= 400:
            CQcc = CQcc[:400, :]
        else:
            CQcc_orig = CQcc
            make_up = int(400/CQcc.shape[0])
            for i in range(make_up-1):
                #print(i)
                CQcc = np.vstack((CQcc,CQcc_orig))
            CQcc = np.vstack((CQcc, CQcc_orig[:(400-(CQcc_orig.shape[0]*make_up)), :]))
        print("??????")
        print(CQcc.shape)
        # Log likelihood ratio computation
        gmm_nat_score[k] = gmm_nat(CQcc)
        #print(gmm_nat_score[k])
        gmm_synt_score[k] = gmm_synt(CQcc)
        #print(gmm_synt_score[k])
        llr_gmm_score[k] = gmm_nat_score[k] - gmm_synt_score[k]
        z_gmm[k] = int(llr_gmm_score[k]>0)
        
        gmm_nat_orig_score[k] = gmm_orig_nat(CQcc[: , :20])
        gmm_synt_orig_score[k] = gmm_orig_synt(CQcc[: , :20])
        llr_gmm_orig_score[k] = gmm_nat_orig_score[k] - gmm_synt_orig_score[k]
        z_orig_gmm[k] = int(llr_gmm_orig_score[k]>0)        
        
        gmm_nat_one_score[k] = gmm_one_nat(CQcc[: , 20:40])
        gmm_synt_one_score[k] = gmm_one_synt(CQcc[: , 20:40])
        llr_gmm_one_score[k] = gmm_nat_one_score[k] - gmm_synt_one_score[k]
        z_one_gmm[k] = int(llr_gmm_one_score[k]>0) 
        
        gmm_nat_two_score[k] = gmm_two_nat(CQcc[: , 40:])
        gmm_synt_two_score[k] = gmm_two_synt(CQcc[: , 40:])
        llr_gmm_two_score[k] = gmm_nat_two_score[k] - gmm_synt_two_score[k]
        z_two_gmm[k] = int(llr_gmm_two_score[k]>0)       
        
        gmm_nat_orig_one_score[k] = gmm_orig_one_nat(CQcc[: , :40])
        gmm_synt_orig_one_score[k] = gmm_orig_one_synt(CQcc[: , :40])
        llr_gmm_orig_one_score[k] = gmm_nat_orig_one_score[k] - gmm_synt_orig_one_score[k]
        z_orig_one_gmm[k] = int(llr_gmm_orig_one_score[k]>0)
        #class_2_label={0:'spoof',1:'bonafide'}
        #if label_2_class[ground_truth.iloc[k].source] != z_gmm[k]:
        #    F_FILE = np.append(F_FILE,ground_truth.iloc[k].file)
            
    k += 1
    

ground_truth['z'] = ground_truth.source.map(lambda x: int(x=='bonafide'))
ground_truth['z_gmm'] = z_gmm
ground_truth['score_gmm'] = llr_gmm_score
ground_truth['gmm_nat_score'] = gmm_nat_score
ground_truth['gmm_synt_score'] = gmm_synt_score
#print(roc_auc_score(ground_truth.z, ground_truth.z_gmm))
print(k,"files processed in",(time.clock()-start_time)/60,"minutes.")

25380
train
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_1004644.wav
orig_shape: (225, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_1006715.wav
orig_shape: (266, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_1007408.wav
orig_shape: (381, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_1017386.wav
orig_shape: (283, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_1025525.wav
orig_shape: (673, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_1029621.wav
orig_shape: (750, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_1030843.wav
orig_shape: (367, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_1033159.wav
orig_shape:

orig_shape: (533, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_1477244.wav
orig_shape: (800, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_1499248.wav
orig_shape: (454, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_1511522.wav
orig_shape: (698, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_1519590.wav
orig_shape: (680, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_1525866.wav
orig_shape: (379, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_1531370.wav
orig_shape: (531, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_1541983.wav
orig_shape: (633, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_1551351.wav
orig_shape: (343, 60)
??????
(400, 60)
/home/wuj

orig_shape: (683, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2042929.wav
orig_shape: (594, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2048941.wav
orig_shape: (487, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2051110.wav
orig_shape: (294, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2056710.wav
orig_shape: (394, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2057403.wav
orig_shape: (196, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2058789.wav
orig_shape: (445, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2086386.wav
orig_shape: (180, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2099563.wav
orig_shape: (171, 60)
??????
(400, 60)
/home/wuj

orig_shape: (320, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2488117.wav
orig_shape: (450, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2488195.wav
orig_shape: (454, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2494677.wav
orig_shape: (317, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2495731.wav
orig_shape: (735, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2503173.wav
orig_shape: (342, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2518848.wav
orig_shape: (528, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2520083.wav
orig_shape: (370, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2534971.wav
orig_shape: (379, 60)
??????
(400, 60)
/home/wuj

orig_shape: (474, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3051141.wav
orig_shape: (364, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3055656.wav
orig_shape: (554, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3055700.wav
orig_shape: (290, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3056559.wav
orig_shape: (257, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3057409.wav
orig_shape: (505, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3060458.wav
orig_shape: (242, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3061594.wav
orig_shape: (285, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3067843.wav
orig_shape: (565, 60)
??????
(400, 60)
/home/wuj

...

orig_shape: (458, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2705905.wav
orig_shape: (234, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2727451.wav
orig_shape: (813, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2745780.wav
orig_shape: (300, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2747022.wav
orig_shape: (385, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2748681.wav
orig_shape: (734, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2762685.wav
orig_shape: (428, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2766626.wav
orig_shape: (729, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2772197.wav
orig_shape: (669, 60)
??????
(400, 60)
/home/wuj

orig_shape: (268, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3258161.wav
orig_shape: (509, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3265392.wav
orig_shape: (624, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3266022.wav
orig_shape: (420, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3267673.wav
orig_shape: (479, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3279709.wav
orig_shape: (440, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3282702.wav
orig_shape: (334, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3286362.wav
orig_shape: (533, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3286480.wav
orig_shape: (634, 60)
??????
(400, 60)
/home/wuj

orig_shape: (230, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3744884.wav
orig_shape: (734, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3745143.wav
orig_shape: (385, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3747538.wav
orig_shape: (446, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3748725.wav
orig_shape: (278, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3751646.wav
orig_shape: (320, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3768404.wav
orig_shape: (256, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3769139.wav
orig_shape: (404, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3778918.wav
orig_shape: (819, 60)
??????
(400, 60)
/home/wuj

orig_shape: (527, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_4177986.wav
orig_shape: (388, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_4180224.wav
orig_shape: (655, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_4183341.wav
orig_shape: (465, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_4184798.wav
orig_shape: (318, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_4187934.wav
orig_shape: (283, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_4197596.wav
orig_shape: (225, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_4200125.wav
orig_shape: (415, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_4205673.wav
orig_shape: (356, 60)
??????
(400, 60)


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
np.save('./score/400_score_train_nat.npy',gmm_nat_score)
np.save('./score/400_score_train_synt.npy',gmm_synt_score)

np.save('./score/400_score_train_orig_nat.npy',gmm_nat_orig_score)
np.save('./score/400_score_train_orig_synt.npy',gmm_synt_orig_score)

np.save('./score/400_score_train_one_nat.npy',gmm_nat_one_score)
np.save('./score/400_score_train_one_synt.npy',gmm_synt_one_score)

np.save('./score/400_score_train_two_nat.npy',gmm_nat_two_score)
np.save('./score/400_score_train_two_synt.npy',gmm_synt_two_score)

np.save('./score/400_score_train_orig_one_nat.npy',gmm_nat_orig_one_score)
np.save('./score/400_score_train_orig_one_synt.npy',gmm_synt_orig_one_score)


In [10]:
ground_truth = train
ground_truth['z'] = ground_truth.source.map(lambda x: int(x=='bonafide'))
print(ground_truth['z'].shape)
np.save('./score/z_train.npy', ground_truth['z'])

(25380,)


In [12]:
# Performance evaluation
ground_truth = train
ground_truth['z'] = ground_truth.source.map(lambda x: int(x=='bonafide'))
ground_truth['z_gmm'] = z_gmm


#实际humans个数
humans = sum(ground_truth.z==1)
print("实际humans个数:", humans)

#实际spoofed个数
spoofed = sum(ground_truth.z==0)
print("实际spoofed个数:", spoofed)

#预测humans个数
pre_humans = sum(ground_truth.z_gmm==1)
print("预测humans个数:", pre_humans)

#预测spoofed个数
pre_spoofed = sum(ground_truth.z_gmm==0)
print("预测spoofed个数:", pre_spoofed)

tp = 0
tn = 0
fp = 0
fn = 0
for i in ground_truth.index:
    if ground_truth.z[i] == 1 and ground_truth.z_gmm[i]==1:
        tp = tp + 1
    elif ground_truth.z[i] == 1 and ground_truth.z_gmm[i]==0:
        fn = fn + 1
    elif ground_truth.z[i] == 0 and ground_truth.z_gmm[i]==0:
        tn = tn + 1
    elif ground_truth.z[i] == 0 and ground_truth.z_gmm[i]==1:
        fp = fp + 1
print("TP_Num:", tp)
print("TN_Num:", tn)
print("FP_Num:", fp)
print("FN_Num:", fn)

tpr = tp/(tp+fn)
fpr = fp/(fp+tn)
tnr = tn/(fp+tn)
fnr = fn/(tp+fn)
acc = (tp+tn)/(tp+fp+tn+fn)
eer = (fpr*fnr)/(fpr+fnr)
print("TPR:", tpr)
print("FPR:", fpr)
print("TNR:", tnr)
print("FNR:", fnr)
print("ROC:", roc_auc_score(ground_truth.z, ground_truth.z_gmm))
print("Accuracy:", acc)
print("EER:", eer)

# humans = z_gmm[z_dvl==0]
# spoofed = z_gmm[z_dvl==1]
# fnr = 100*(1-(humans<thr).sum()/len(humans))
# fpr = 100*(1-(spoofed>=thr).sum()/len(spoofed))
# print("ROC AUC score:", roc_auc_score(z_dvl,z_gmm))
# print("False negative rate %:", fnr)
# print("False positive rate %:", fpr)
# print("EER %: <=", (fnr+fpr)/2)

实际humans个数: 2580
实际spoofed个数: 22800
预测humans个数: 7324
预测spoofed个数: 18056
TP_Num: 1370
TN_Num: 16846
FP_Num: 5954
FN_Num: 1210
TPR: 0.5310077519379846
FPR: 0.261140350877193
TNR: 0.738859649122807
FNR: 0.4689922480620155
ROC: 0.6349337005303958
Accuracy: 0.7177304964539007
EER: 0.16774049041987163


In [62]:
method = {}
for index, row in ground_truth.iterrows():
    if row['z'] != row['z_gmm']:
        if row['method'] in method:
            method[row['method']] = method[row['method']] + 1
        else:
            method[row['method']] = 1
print(method)

{'-': 6757, 'A07': 17, 'A16': 18, 'A10': 28, 'A12': 21, 'A11': 25, 'A13': 6, 'A15': 4, 'A18': 13, 'A17': 14, 'A19': 39, 'A08': 2}


In [47]:
ground_truth

,folder,file,method,source,z,z_gmm,score_gmm
2597,D1,D1_1002598,human,human,1,1.0,1.447147
2598,D1,D1_1002599,human,human,1,1.0,2.336380
2599,D1,D1_1002600,human,human,1,1.0,3.082637
2600,D1,D1_1002601,human,human,1,1.0,3.716948
2601,D1,D1_1002602,human,human,1,1.0,0.914256
...,...,...,...,...,...,...,...
7767,D3,D3_1007768,S1,spoof,0,0.0,-2.641017
7768,D3,D3_1007769,S2,spoof,0,0.0,-0.571958
7769,D3,D3_1007770,S3,spoof,0,0.0,-5.441518
7770,D3,D3_1007771,S4,spoof,0,0.0,-4.762759


### EER computation

Adjust the threshold $thr$ to reduce $FNR-FPR$ for a more accurate estimate of the $EER$.

The Equal Error Rate ($EER$) is the value where the false negative rate ($FNR$) equals the false positive rate ($FPR$). It's an error metric commonly used to characterize biometric systems.

In [47]:
thr = -.115
pz = llr_gmm_score
spoofed = pz[np.array(ground_truth.z)==1]
humans = pz[np.array(ground_truth.z)==0]
fnr = 100*(humans>thr).sum()/len(humans)
fpr = 100*(spoofed<=thr).sum()/len(spoofed)
print("False negative vs positive rates %:", fnr, fpr)
print("FNR - FPR %:", fnr-fpr)
if np.abs(fnr-fpr) <.25:
    print("EER =", (fnr+fpr)/2,"%")
else:
    print("EER ~", (fnr+fpr)/2,"%")

False negative vs positive rates %: 0.3345997002544352 99.53318745441284
FNR - FPR %: -99.19858775415841
EER ~ 49.933893577333635 %
